In [ ]:
# how use throw method example
import asyncio
async def f():
    try:
        while True:
            print('I  work')
            await asyncio.sleep(0)
    except asyncio.CancelledError:
        print('I was canceled')


coro = f()
coro.send(None)
coro.send(None)
coro.send(None)
coro.throw(asyncio.CancelledError)

In [ ]:
# Async context manager
async def _get_conn(*args):
    pass


class Connection:
    """ Async context manager """

    def __init__(self, host, port):
        self.host = host
        self.port = port

    async def __aenter__(self):
        self.conn = await _get_conn(self.host, self.port)
        return conn

    async def __aexit__(self, exc_type, exc, tb):
        await self.conn.close()


async with Connection('localhost', 9001) as conn:
    pass



In [ ]:
"""
async context manager and async generator
"""
from contextlib import asynccontextmanager


async def download_webpage(url):
    pass


async def update_stats(url):
    pass


def process(data):
    pass


@asynccontextmanager
async def web_page(url):
    data = await download_webpage(url)
    yield data
    await update_stats(url)

async with web_page('google.com') as data:
    process(data)


In [ ]:
"""
This is example how run blocking code into async functions.
download_webpage and update_stats are blocking
"""
@asynccontextmanager
async def web_page(url):
    loop = asyncio.get_event_loop()
    data = await loop.run_in_executor(None, download_webpage, url)
    yield data
    await loop.run_in_executor(None, update_stats, url)


async with web_page('google.com') as data:
    process(data)

In [ ]:
"""
Async iterator and async for in
"""
import asyncio


async def save_text_in_db(v):
    print(f'начали сохранение => {v}')
    await asyncio.sleep(1)
    print(f'закончили запись в БД => {v}')


async def main():
    urls = [1, 2, 3, 4, 5]
    async for value in OneAtATime(urls):
        await save_text_in_db(value)


class OneAtATime:
    def __init__(self, urls):
        self.urls = urls

    def __aiter__(self):
        self.urls = iter(self.urls)
        return self

    async def __anext__(self):
        try:
            i = next(self.urls)
        except StopIteration:
            raise StopAsyncIteration

        print(f'make request in DB => {i}')
        await asyncio.sleep(i)
        return i


asyncio.run(main())

In [ ]:
"""
Async generator
"""
async def main2():
    urls = [1, 2, 3, 4, 5]
    async for value in generator(urls):
        await save_text_in_db(value)


async def generator(urls):
    for i in urls:
        print(f'сделали запрос в бд {i}')
        await asyncio.sleep(i)
        yield i


asyncio.run(main2())

In [ ]:
"""
Comprehension
"""
async def main_comprehension():
    results = [await f(x) async for f, x in factory(3)]
    print('results = ', results)


In [3]:
"""
И всё таки сделаем одновременные запросы
"""
from datetime import datetime
import asyncio


async def save_text_in_db(v):
    print(f'начали сохранение => {v}')
    await asyncio.sleep(1)
    print(f'закончили запись в БД => {v}')


async def main3():
    urls = [1, 2, 3, 4, 5]
    coros = [get_from_db(i) for i in urls]
    result = await asyncio.gather(*coros)
    print(result)
    return result


async def get_from_db(i):
    print(f'Делаем долгий запрос => {i}')
    await asyncio.sleep(i)
    print(f'Закончили запрос => {i}')
    await save_text_in_db(i)
    print(f'done => {i}')
    return f'done => {i}'


start = datetime.now()
print(start)
asyncio.run(main3())
finish = datetime.now()
print(finish)
print(f'time => {finish - start}')

2020-10-20 16:22:53.879176


RuntimeError: asyncio.run() cannot be called from a running event loop

In [ ]:
"""
Как завершить все таки в случае краша сервера
import asyncio
"""
async def main():
    while True:
        print('<Your app is running>')
        await asyncio.sleep(1)

if __name__ == '__main__':
    loop = asyncio.get_event_loop()
    task = loop.create_task(main())
    try:
        loop.run_until_complete(task)
    except KeyboardInterrupt:
        print('Got signal: SIGINT, shutting down.')

    tasks = asyncio.all_tasks(loop=loop)
    for t in tasks:
        t.cancel()

    group = asyncio.gather(*tasks, return_exceptions=True)
    loop.run_until_complete(group)
    loop.close()
